<a href="https://colab.research.google.com/github/xahram/Sci-Bert/blob/main/Sci_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First step is to load the NIPS data that is uploaded in the Google Drive

In [ ]:
# Mount the google drive folder into the directory to access files

from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [ ]:
# Load all necessary libraries

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer

nltk.download("punkt")
nltk.download('stopwords')




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load the NIPS dataset from the drive

nips_papers_df = pd.read_csv('/gdrive/My Drive/Master_dataset/papers.csv')  
nips_papers_df.head()

nips_papers = nips_papers_df.infer_objects()

nips_papers.dtypes

nips_papers["year"] = pd.to_datetime(nips_papers["year"], format="%Y")
# nips_papers['year'] = nips_papers['year'].dt.year
nips_papers.sort_values(by='year')

print(nips_papers.dtypes)

max(nips_papers["year"])
min(nips_papers["year"])

nips_papers = nips_papers.sort_values(by = "year")


id                     int64
year          datetime64[ns]
title                 object
event_type            object
pdf_name              object
abstract              object
paper_text            object
dtype: object


In [ ]:
from os import close
# Slice Data Frame by 3 year interval


# print(len(nips_papers))

# Partition/Group Papers into df by the interval/freq of 3 years, closed = left to start combinbing from the 1987
nips_papers_3y_grouped = nips_papers.groupby(pd.Grouper(key='year', freq='3Y', sort=True, closed="left"))



# Save partitions in the Dictionary format with 10 intervals
nips_papers_partitions = {}
initial_partition_id = 0
for i, g  in nips_papers_3y_grouped:
    nips_papers_partitions[initial_partition_id] = g
    initial_partition_id = initial_partition_id + 1


print(nips_papers_partitions)
# nips_papers_three_year_partition[0].tail()


#for i, g in nips_papers.groupby(pd.Grouper(key=nips_papers["year"], freq='A')):
#     print(g)







{0:        id       year                                              title  \
0       1 1987-01-01  Self-Organization of Associative Database and ...   
328    13 1987-01-01   Temporal Patterns of Activity in Neural Networks   
6853   72 1987-01-01  Ensemble' Boltzmann Units have Collective Comp...   
6743   71 1987-01-01  Centric Models of the Orientation Map in Prima...   
6632   70 1987-01-01  On the Power of Neural Networks for Solving Ha...   
...   ...        ...                                                ...   
1650  250 1989-01-01                               Optimal Brain Damage   
1661  251 1989-01-01  A Self-organizing Associative Memory System fo...   
1672  252 1989-01-01  Can Simple Cells Learn Curves? A Hebbian Model...   
1683  253 1989-01-01  Subgrouping Reduces Complexity and Speeds Up L...   
1638  249 1989-01-01  Neural Network Analysis of Distributed Represe...   

     event_type                                           pdf_name  \
0           NaN  1-self-o

In [ ]:
# IMPORT word2phrase to create bigrams and unigrams
!git clone https://github.com/travisbrady/word2phrase.git

Cloning into 'word2phrase'...
remote: Enumerating objects: 93, done.
remote: Total 93 (delta 0), reused 0 (delta 0), pack-reused 93
Unpacking objects: 100% (93/93), done.


# FIRST TIME WINDOW SCI-BERT

In [ ]:
# CONVERT First Time Window's Paper content into list

paper_contents_list = nips_papers_partitions[0]["paper_text"].tolist()

# Concatenate all the papers of the first time window
#### MEASURE THE EXECUTION TIME FOR RUNNING THE CONCATENATION CODE
import time

start = time.time()

first_time_window_all_paper_content = " ".join(paper_contents_list)[:20000]

end = time.time()
print(end - start)

len(first_time_window_all_paper_content)

0.0031838417053222656


20000

In [ ]:
from google.colab import files

# files.upload()


In [ ]:
# IMPORT and Download NLTK corpus dependencies
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:

# Preprocessing of papers: -? Stopwords removal...

# papers_text_list = nips_papers_df["PaperText"].tolist()

# first_article_word_tokens = nltk.word_tokenize(papers_text_list[0])

# stopwords = set(stopwords.words("english"))

# cleaned_first_article_word_tokens = [w for w in first_article_word_tokens if w not in stopwords]

# print(cleaned_first_article_word_tokens)
# %load word2phrase/word2phrase.py




# natural language processing: n-gram ranking
import re
import unicodedata
from nltk.corpus import stopwords


def basic_clean(text):
  """
  A simple function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """
  wnl = nltk.stem.WordNetLemmatizer()
  stopwords = nltk.corpus.stopwords.words('english') 
  text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
  words = re.sub(r'[^\w\s]', '', text).split()
  return [wnl.lemmatize(word) for word in words if word not in stopwords]




words = basic_clean(first_time_window_all_paper_content)


# Get number similar items and their frequencies with value_count and convert it to dictionary
bi_grams_unfiltered = pd.Series(nltk.ngrams(words, 2)).value_counts().to_dict()


# Import Islice from Itertool for getting n-items from the dictionary
from itertools import islice

# Get list of Bi_grams Key-value pairs
bi_grams_freq_list = list(bi_grams_unfiltered.items())[0:40]


# Format Bi_grams using _ 
# Convert list of two dimensional tuples into single ones e.g ((neural,network), 2) -> (neural_network,2)

formatted_bi_grams = [(" ".join(bi_gram[0]), bi_gram[1])  for bi_gram in bi_grams_freq_list]
formatted_bi_grams


bi_gram_features = {}
for tup in formatted_bi_grams:
  bi_gram_features[tup[0]] = tup[1]

bi_gram_features
# top_freq_bi_grams = islice(bi_grams_freq_list,40)
# list(top_freq_bi_grams)

# for bi_gram in nltk.ngrams(words, 2):
#     print(bi_gram)











{'algorithm 3': 3,
 'associative database': 13,
 'autonomous mobile': 3,
 'avoiding movement': 4,
 'black point': 8,
 'camera image': 6,
 'define accordingly': 3,
 'define following': 3,
 'define x': 3,
 'dx x': 8,
 'e x': 9,
 'estimation error': 6,
 'even though': 3,
 'fig 1': 4,
 'fig 3': 4,
 'fig 5': 5,
 'follows algorithm': 3,
 'image x': 6,
 'learning machine': 9,
 'letter recognition': 3,
 'mobile robot': 3,
 'n n': 5,
 'neural network': 8,
 'number sample': 5,
 'observed sample': 4,
 'obstacle avoiding': 4,
 'parameter type': 4,
 'position data': 4,
 'position identification': 5,
 'recognition rate': 5,
 'recursive type': 4,
 'selforganization associative': 5,
 'structure f': 4,
 'type learning': 3,
 'window image': 4,
 'x accordingly': 3,
 'x define': 4,
 'x e': 8,
 'x x': 23,
 'xn yin': 3}

## Step 1 : Pre Processing - Custom Stop Word And  Number Regex

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re



first_time_window_all_paper_content = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", first_time_window_all_paper_content)

# Remove words with length less than 3 
# https://stackoverflow.com/questions/24332025/remove-words-of-length-less-than-4-from-string

first_time_window_all_paper_content = re.sub(r'\b\w{1,2}\b', '',first_time_window_all_paper_content)






In [ ]:
# Custom Stopwords List for Scientific Literature 

from sklearn.feature_extraction import text

path_to_stop_words = '/gdrive/My Drive/Master_dataset/stopwords_10000_most_frequent_filtered.txt'

with open(path_to_stop_words, "r") as file1:
    FileasList = file1.readlines()


stopwords = [s.strip('\n') for s in FileasList]
print(len(stopwords))


scientific_literature_stopwords = text.ENGLISH_STOP_WORDS.union(stopwords)

len(scientific_literature_stopwords)


9954


9958

## Step 2 - Create Bag Of Unigrams and BIgrams

In [ ]:
n_features = 200 
sent_tokens  = nltk.sent_tokenize(first_time_window_all_paper_content)

In [ ]:
## Calculate Bag Of Bi_grams

vect = DictVectorizer()


vectorizer = CountVectorizer(stop_words=scientific_literature_stopwords,ngram_range=(2,2), 
                             max_features=n_features,lowercase=True, max_df=0.6)
# vectorizer_uni_grams = CountVectorizer(stop_words='english',ngram_range=(1,1), max_features=n_features)


x_train = vectorizer.fit_transform(sent_tokens)

df_bow_bi_grams = pd.DataFrame(x_train.toarray(),columns=vectorizer.get_feature_names())
df_bow_bi_grams


bigram_candidate_keywords = vectorizer.get_feature_names()




# df_bow_bi_grams = pd.DataFrame(x_train.toarray(),columns=vect.get_feature_names())
# df_bow_bi_grams


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['adj', 'ain', 'anglo', 'aren', 'cf', 'ch', 'col', 'cong', 'couldn', 'didn', 'doesn', 'false', 'fashioned', 'father', 'ff', 'friend', 'ft', 'gen', 'george', 'girl', 'god', 'hadn', 'hasn', 'haven', 'icel', 'imp', 'isn', 'king', 'lady', 'law', 'majesty', 'michael', 'mother', 'mustn', 'ne', 'needn', 'oe', 'pp', 'prof', 'shan', 'shouldn', 'son', 'sq', 'th', 've', 'viz', 'wasn', 'weren', 'wife', 'woman', 'wouldn'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
## Calculate Bag Of uni_grams


vectorizer_uni = CountVectorizer(stop_words=scientific_literature_stopwords,ngram_range=(1,1), 
                             max_features=n_features,lowercase=True, min_df=1)
# vectorizer_uni_grams = CountVectorizer(stop_words='english',ngram_range=(1,1), max_features=n_features)


x_train_uni = vectorizer_uni.fit_transform(sent_tokens)

df_bow_uni_grams = pd.DataFrame(x_train_uni.toarray(),columns=vectorizer_uni.get_feature_names())
df_bow_uni_grams


uni_gram_candidate_keywords = vectorizer_uni.get_feature_names()



# df_bow_uni_grams = pd.DataFrame(x_train.toarray(),columns=vect.get_feature_names())
# df_bow_uni_grams


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['adj', 'ain', 'anglo', 'aren', 'cf', 'ch', 'col', 'cong', 'couldn', 'didn', 'doesn', 'false', 'fashioned', 'father', 'ff', 'friend', 'ft', 'gen', 'george', 'girl', 'god', 'hadn', 'hasn', 'haven', 'icel', 'imp', 'isn', 'king', 'lady', 'law', 'majesty', 'michael', 'mother', 'mustn', 'ne', 'needn', 'oe', 'pp', 'prof', 'shan', 'shouldn', 'son', 'sq', 'th', 've', 'viz', 'wasn', 'weren', 'wife', 'woman', 'wouldn'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# vectorizer_uni.vocabulary_

In [ ]:
feature_frequencies = x_train.toarray().sum(axis=0)


bi_gram_keyword_frequency = {}

for index, keyword in enumerate(vectorizer.get_feature_names()):
  bi_gram_keyword_frequency[keyword]=feature_frequencies[index]


feature_frequencies_uni = x_train_uni.toarray().sum(axis=0)

print(feature_frequencies_uni)

uni_gram_keyword_frequency = {}

for index,keyword in enumerate(vectorizer_uni.get_feature_names()):
  uni_gram_keyword_frequency[keyword]=feature_frequencies_uni[index]




# import nltk
# from nltk import word_tokenize
# from nltk.util import ngrams
# from collections import Counter


# n_features = 200 
# word_tokens  = nltk.word_tokenize(first_time_window_all_paper_content)

# uni_grams = ngrams(word_tokens,1)
# bi_grams = ngrams(word_tokens,2)

# Counter(uni_grams)


[ 1  1  2  1  1  1  1  1  1  1  2  1  2  8  3 13  3  2  3  1  1  1  1 11
  3  1  1  1  1  2  2  1  3  1  2  1  3  1  1  3  1  1  3  1  2  2  1  1
  3  1 12  7  1  1  1  1  2  2  1  2  1  3  2  2  3  2  2  2 21  4  2  3
  3  2  2  2  2  2  2  5  2  2  2  8  4  2  1  2  2  1  1  1  1  2  1  3
  2  2  6  8  1  6  1  1  1  2  2  1  1 10  3  1  2  2  1  1  4  4  2  1
  1  1  1  2  2  6  1  2  1  2  1  1  1  1  1  4  3  1  4  1  1  1  1  1
  3  1  1  1  1  2 21  1  1  1  1 13  1  1  1  1  1  2  1  1  1  4  1  1
  3  1  1  1  1  2  1  1  1  1  3  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  3  1  5]


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Join BOth Bag of Unigrams and Bigrams

bag_of_candidate_keywords = {**bi_gram_keyword_frequency, **uni_gram_keyword_frequency}


# Sort the bag of words
candidate_keywords_bow = sorted(bag_of_candidate_keywords.items(), key=lambda item: item[1], reverse=True)
len(candidate_keywords_bow)

candidate_keywords_bow

[('fig', 21),
 ('robot', 21),
 ('associative', 13),
 ('samples', 13),
 ('database', 12),
 ('camera', 11),
 ('operator', 10),
 ('associative database', 8),
 ('algorithm', 8),
 ('letter', 8),
 ('neural', 8),
 ('databases', 7),
 ('neural networks', 6),
 ('networks', 6),
 ('nodes', 6),
 ('problem', 6),
 ('associative databases', 5),
 ('increases', 5),
 ('yin', 5),
 ('finite', 4),
 ('levels', 4),
 ('parameter', 4),
 ('parameters', 4),
 ('recognizer', 4),
 ('recursive', 4),
 ('structures', 4),
 ('autonomous mobile', 3),
 ('constructing associative', 3),
 ('mobile robot', 3),
 ('samples increases', 3),
 ('approaches', 3),
 ('assumed', 3),
 ('autonomous', 3),
 ('candidates', 3),
 ('compression', 3),
 ('computes', 3),
 ('constructing', 3),
 ('contrarily', 3),
 ('criterion', 3),
 ('dimension', 3),
 ('english', 3),
 ('frequency', 3),
 ('furthermore', 3),
 ('mobile', 3),
 ('organizing', 3),
 ('recognizing', 3),
 ('reset', 3),
 ('subject', 3),
 ('teaches', 3),
 ('white', 3),
 ('approaches samples',

In [ ]:
candidate_keywords_bow = candidate_keywords_bow[:100]
candidate_keywords_bow = [keyword[0] for keyword in candidate_keywords_bow] 
candidate_keywords_bow

['fig',
 'robot',
 'associative',
 'samples',
 'database',
 'camera',
 'operator',
 'associative database',
 'algorithm',
 'letter',
 'neural',
 'databases',
 'neural networks',
 'networks',
 'nodes',
 'problem',
 'associative databases',
 'increases',
 'yin',
 'finite',
 'levels',
 'parameter',
 'parameters',
 'recognizer',
 'recursive',
 'structures',
 'autonomous mobile',
 'constructing associative',
 'mobile robot',
 'samples increases',
 'approaches',
 'assumed',
 'autonomous',
 'candidates',
 'compression',
 'computes',
 'constructing',
 'contrarily',
 'criterion',
 'dimension',
 'english',
 'frequency',
 'furthermore',
 'mobile',
 'organizing',
 'recognizing',
 'reset',
 'subject',
 'teaches',
 'white',
 'approaches samples',
 'criterion operator',
 'databases robot',
 'head letter',
 'input output',
 'operator teaches',
 'organizing associative',
 'robot eyesight',
 'subject constructing',
 'white fig',
 'yin yin',
 '18m',
 '400',
 '800',
 'attains',
 'coefficients',
 'compactn

## Step 2 : After Generating BOW_BiGrams, Next Step is to generate BERT based Embedding

Install Pretrained interface of pytorch for BERT


In [ ]:
!pip install pytorch-pretrained-bert transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 123 kB 29.4 MB/s 
     |████████████████████████████████| 4.7 MB 62.1 MB/s 
     |████████████████████████████████| 132 kB 66.6 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
     |████████████████████████████████| 6.6 MB 65.3 MB/s 
     |████████████████████████████████| 101 kB 7.7 MB/s 
     |████████████████████████████████| 9.0 MB 48.6 MB/s 
     |████████████████████████████████| 79 kB 4.4 MB/s 
     |████████████████████████████████| 139 kB 60.1 MB/s 
     |████████████████████████████████| 127 kB 54.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dep

In [ ]:
# importing libraries for BERT based embedding
 
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 22119798.86B/s]


## Step 2.1 - Sample Test for Using BERT is Performed Below // Expand To see guide on Generating BERT embeddings For Sentences 

In [ ]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

example_text = 'I will watch Memento tonight'

bert_input = tokenizer(example_text,padding='max_length', max_length = 10, 
                       truncation=True, return_tensors="pt")

print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

tensor([[  101,  1045,  2097,  3422,  2033, 23065,  3892,   102,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [ ]:
print(bert_input)
tokenizer.decode(bert_input.input_ids[0])

{'input_ids': tensor([[  101,  1045,  2097,  3422,  2033, 23065,  3892,   102,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])}


'[CLS] i will watch memento tonight [SEP] [PAD] [PAD]'

In [ ]:
bert_input = tokenizer.encode_plus(["Some random sentence 1", "Some random sentence 2"])
tokenizer.decode(bert_input.input_ids)
bert_input

{'input_ids': [101, 100, 100, 102], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [ ]:
# Create A Utility Class for generating Bert Based Embeddding for the every sentence

sentence_tokens_first_time_window = sent_tokens
bert_tokens_first_time_window = [tokenizer(text,  padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in sentence_tokens_first_time_window]



In [ ]:

bert_tokens_test = bert_tokens_first_time_window[:10]

decoded_sentences = [tokenizer.decode(bert_token.input_ids[0]) for bert_token in bert_tokens_test]
decoded_sentences

['[CLS] self - organization associative database and its applications hisashi suzuki and suguru arimoto osaka university, toyonaka, osaka 560, japan abstract efficient method self - organizing associative databases proposed together with applications robot eyesight systems. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [ ]:
# Alternative 2 : Generating BERt Embeddings using BERT Encode method

bert_input_ids = tokenizer.batch_encode_plus(sentence_tokens_first_time_window, return_token_type_ids=False, return_attention_mask=False)






In [ ]:
bert_input_ids.input_ids[0:10]

[[101,
  2969,
  1011,
  3029,
  4632,
  10085,
  2401,
  6024,
  7809,
  1998,
  2049,
  5097,
  2010,
  12914,
  14278,
  1998,
  10514,
  27390,
  2226,
  10488,
  15319,
  13000,
  2118,
  1010,
  9121,
  7856,
  2912,
  1010,
  13000,
  21267,
  1010,
  2900,
  10061,
  8114,
  4118,
  2969,
  1011,
  10863,
  4632,
  10085,
  2401,
  6024,
  17881,
  3818,
  2362,
  2007,
  5097,
  8957,
  2159,
  18743,
  3001,
  1012,
  102],
 [101, 1996, 3818, 17881, 2064, 5482, 2151, 7953, 2007, 2070, 6434, 1012, 102],
 [101,
  1996,
  2034,
  2431,
  2112,
  6594,
  1010,
  9896,
  2969,
  1011,
  3029,
  3818,
  1012,
  102],
 [101, 2013, 7814, 8051, 1010, 7137, 2047, 2806, 15756, 2897, 1012, 102],
 [101,
  1996,
  3732,
  2431,
  2112,
  1010,
  10439,
  19341,
  8553,
  2192,
  15773,
  3661,
  5038,
  1998,
  2008,
  8392,
  4684,
  8957,
  2291,
  2024,
  7645,
  1012,
  102],
 [101, 4955, 2292, 12375, 1024, 1011, 1009, 2445, 1012, 102],
 [101,
  2182,
  1010,
  10713,
  10709,
  2275,


In [ ]:
# Compute Cosine Similarity
import numpy as np

bert_tes =  [ bert_token.input_ids for bert_token in bert_tokens_test]
# np.dot(bert_tes[0], bert_tes[1])

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances




pairwise_similarities=cosine_similarity(bert_tes[0], bert_tes[0])
pairwise_similarities


# bert_tes[0][0]

array([[1.]])

## TEST 1 - Creating BERT Embedding For NIPS paper and Calculate Cosine Similarity

In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.8 MB/s 
     |████████████████████████████████| 1.2 MB 49.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=9c404046861ab8edc9074ebb6ea25d8b975931349345e06c9ae39a70e611604c
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("distilbert-base-nli-mean-tokens")
bert_sentence_embedding_first_time_window = model.encode(sent_tokens[:100])

keyword_feaature_bert_embedding = model.encode(bigram_candidate_keywords)


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/450 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
print(len(keyword_feaature_bert_embedding))

keyword_feaature_bert_embedding

200


array([[-0.26855186, -0.6045678 ,  0.47657362, ..., -0.52285945,
        -0.3049894 , -0.925612  ],
       [-0.73744106, -0.8641267 ,  0.6937428 , ..., -1.3673078 ,
         0.10443189, -0.5714228 ],
       [-1.0187142 , -0.70818657,  0.36998767, ..., -1.0079896 ,
        -0.30755278, -0.0738718 ],
       ...,
       [ 0.0975033 , -0.07088516,  0.26368952, ..., -0.05047144,
        -0.1023604 , -0.8592203 ],
       [-0.4740122 , -1.0899329 ,  0.67605585, ...,  0.71893084,
        -0.27680606,  0.14375247],
       [ 0.11279758, -0.1743159 ,  0.57081133, ..., -0.49074906,
        -0.00307617,  0.5477061 ]], dtype=float32)

In [ ]:
# Calculate Cosine Similarity 

from sklearn.metrics.pairwise import cosine_similarity

top_n = 300

distances = cosine_similarity(keyword_feaature_bert_embedding, keyword_feaature_bert_embedding)

print(len(distances))
# keywords = [keyword_feaature_bert_embedding[index] for index in distances.argsort()[0][-top_n:]]
keywords = [bigram_candidate_keywords[index] for index in distances.argsort()[0][-top_n:]]
keywords

top_300_embeddings_with_similar_cos_similarity = [keyword_feaature_bert_embedding[index] for index in distances.argsort()[0][-top_n:]]




200


In [ ]:
top_300_embeddings_with_similar_cos_similarity[:1]

[array([-1.60771585e+00,  7.43093863e-02,  3.71663332e-01, -6.22159123e-01,
        -7.09010720e-01, -1.06052828e+00,  6.97452128e-01,  8.18166256e-01,
        -1.45838216e-01, -4.33979601e-01,  6.26473427e-01,  6.41432822e-01,
        -3.79246473e-02,  7.04947263e-02, -1.51248157e-01, -7.14931667e-01,
        -5.47982216e-01,  1.19002936e-02,  4.99034941e-01, -4.76399243e-01,
        -6.62177026e-01, -3.17990541e-01,  1.59147888e-01,  7.89199531e-01,
        -5.46395779e-01,  5.71101606e-01, -4.06214707e-02,  1.02923703e+00,
        -4.70834315e-01,  3.49680573e-01, -2.36328110e-01, -5.63173234e-01,
        -1.85266733e-01,  4.49257910e-01,  8.55134666e-01,  1.34226307e-01,
         7.19548464e-01, -3.23993206e-01, -6.14798248e-01,  6.88172400e-01,
         6.33144617e-01,  1.72627181e-01,  1.31822512e-01,  1.39601916e-01,
        -1.04471362e+00,  2.99834818e-01, -5.82042992e-01,  6.08406186e-01,
        -1.59332681e+00, -2.03079000e-01,  1.62999537e-02,  2.71307647e-01,
         1.3

In [ ]:
# Apply K-NN to calculate the Nearest neighbor of keywords

from sklearn.neighbors import NearestNeighbors




keyword_feaature_bert_embedding = keyword_feaature_bert_embedding[0:300]
model = NearestNeighbors(n_neighbors=10,
                         metric='cosine',
                         algorithm='brute',
                         n_jobs=-1)
first_time_window_candidate_nn = model.fit(keyword_feaature_bert_embedding)

# This gives a matrix of 300, 10 -> 300 keywords/arrays with each array containing 10 elements
# representing its nearest neighbor
 
distance, indeces = first_time_window_candidate_nn.kneighbors(keyword_feaature_bert_embedding)

# Give the indeces of the neighbors that are nearest to the word under consideration
## Here we have 300 indeces each with 10 sub array elements, representing it's 10 nearest keywords



In [ ]:
indeces[0]

array([  0,  68, 192,   1,  44,  67, 131, 100, 132,  63])

In [ ]:
keywords


['robot database',
 'robot reset',
 'physics designing',
 'databases robot',
 'speciiications 20dot',
 'operator teaches',
 'problem designing',
 'robot roan',
 'robot operator',
 'networks structures',
 'robot 30kg',
 'osaka 560',
 'operator brushes',
 'robot arbitrarily',
 'robot camera',
 'camera 55deg',
 'randomly algorithm',
 'rewriting free',
 'suzuki suguru',
 'nodes 145',
 'methodologies problem',
 'robot identifies',
 'rmbi robot',
 'laboratories fig',
 '560 organizing',
 'osaka toyonaka',
 'neural networks',
 'operator obstacles',
 'roan uni',
 'builds organizing',
 'networks approximations',
 'robot recalling',
 'toyonaka osaka',
 'associative databases',
 'mobile robot',
 'brushes obstacles',
 'uni fig',
 'associative database',
 'terminal node',
 'variables nodes',
 'problem criterion',
 'sampled randomly',
 'stools handcarts',
 'controls 3km',
 'rewrites correlated',
 'robot eyesight',
 'problem compression',
 'subject constructing',
 'minimization problem',
 'yin yin',
 

In [ ]:
keyword_feaature_bert_embedding


keywords_np = np.array(keywords)
bigram_candidate_keywords_nparray = np.array(bigram_candidate_keywords) 

bigram_candidate_keywords_nparray

keywords_np

tup_nearest_neighbor = []
for index, candidate_keyword in enumerate(keywords_np):
    # Take the current index of the keyword and get the list of 10 nearest index from KNN algorithm
    nearest_neighbors_indeces_of_current_keyword = indeces[index]

    # Filter the keyword list using the list of indeces obtained in previous step
    nearest_keywords = keywords_np[nearest_neighbors_indeces_of_current_keyword]

    # Create tuple with first element as the keyword for current iteration and 2nd element as list of its nearest neighbors
    tup_nearest_neighbor.append((candidate_keyword, nearest_keywords))


tup_nearest_neighbor

[('robot database',
  array(['robot database', 'sampl fig', 'parallelogram maximal',
         'robot reset', 'rewrites correlated', 'problem identifying',
         'recognizer scans', 'reset periodically', 'units computing',
         'white fig'], dtype='<U26')),
 ('robot reset',
  array(['robot reset', 'rewrites correlated', 'databases robot',
         'robot database', 'learns landscapes', 'white fig',
         'camera inquires', 'physics designing', 'typographic english',
         'compression admittable'], dtype='<U26')),
 ('physics designing',
  array(['physics designing', 'nodes memorize', 'reproduces criterion',
         'tly slitting', 'robot arbitrarily', 'neural networks',
         'mobile robot', 'osaka toyonaka', 'brushes obstacles',
         'head letter'], dtype='<U26')),
 ('databases robot',
  array(['databases robot', 'robot reset', 'rewrites correlated',
         'nand nodes', 'robot database', 'compression admittable',
         'learns landscapes', 'white fig', 'match

## TEST 2 - SENTENCE BERT : TEST/SAMPLE For 1 TIME WINDOW

In [ ]:
# Install SBERT
!pip install sentence_transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Creating Custom Network From Scratch

# https://www.sbert.net/docs/training/overview.html

from sentence_transformers import SentenceTransformer, models


word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Pretrained Model // Comment this Cell to use the Custom Model in the above cell

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')


In [ ]:
# Generate Word Embeddings Using SBERT encode
doc_embedding = model.encode(sent_tokens[:100])
candidate_keyword_embedding = model.encode(candidate_keywords_bow)





In [ ]:
len(candidate_keyword_embedding[0])

768

In [ ]:
# Create Textual Similarity b/w corpus and candidate keywords
from sentence_transformers import util

cosine_scores = util.cos_sim(doc_embedding, candidate_keyword_embedding)

print(len(candidate_keyword_embedding))

top_n = 100
keywords = [candidate_keywords_bow[index] for index in cosine_scores.argsort()[0][-top_n:]]
keywords


100


['incorrect',
 'negligible',
 'free',
 'contrarily',
 'furthermore',
 'problem',
 'finite',
 'implicitly',
 'assumed',
 'eternally',
 'subject',
 'frequency',
 'attains',
 '400',
 'observes',
 'dissimilari',
 'increases',
 'denote',
 'head',
 'levels',
 'letter',
 'fig',
 'recognizing',
 'learns',
 '18m',
 'defines',
 'parameter',
 'hand',
 'camera',
 'min',
 'juts',
 'approaches',
 'preliminarily',
 'compactness',
 'samples increases',
 'parameters',
 'obstacles',
 'criterion',
 'output',
 'luminance',
 'white',
 'reset',
 'yin',
 '800',
 'compression',
 'converges',
 'nand',
 'converged',
 'eyesight',
 'structures',
 'head letter',
 'coefficients',
 'handwritten',
 'recognizer',
 'passageway',
 'samples',
 'recursive',
 'computes',
 'white fig',
 'nodes',
 'dimension',
 'operator',
 'input',
 'english',
 'autonomous',
 'associative',
 'yin yin',
 'global',
 'input output',
 'approaches samples',
 'candidates',
 'constructing',
 'organizing',
 'subject constructing',
 'networks',
 'te

In [ ]:
# Find Pairs with highest cosine similarity

pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(len(doc_embedding[i]), len(candidate_keyword_embedding[j]), pair['score']))

768 		 768 		 Score: 0.9355
768 		 768 		 Score: 0.9143
768 		 768 		 Score: 0.9076
768 		 768 		 Score: 0.9036
768 		 768 		 Score: 0.8943
768 		 768 		 Score: 0.8874
768 		 768 		 Score: 0.8805
768 		 768 		 Score: 0.8757
768 		 768 		 Score: 0.8713
768 		 768 		 Score: 0.8704


In [ ]:
# Apply K-NN to calculate the Nearest neighbor of keywords

from sklearn.neighbors import NearestNeighbors




keyword_feaature_bert_embedding = candidate_keyword_embedding
model = NearestNeighbors(n_neighbors=10,
                         metric='cosine',
                         algorithm='brute',
                         n_jobs=-1)
first_time_window_candidate_nn = model.fit(keyword_feaature_bert_embedding)

# This gives a matrix of 300, 10 -> 300 keywords/arrays with each array containing 10 elements
# representing its nearest neighbor
 
distance, indeces = first_time_window_candidate_nn.kneighbors(keyword_feaature_bert_embedding)

# Give the indeces of the neighbors that are nearest to the word under consideration
## Here we have 300 indeces each with 10 sub array elements, representing it's 10 nearest keywords



In [ ]:
keyword_feaature_bert_embedding


keywords_np = np.array(keywords)
bigram_candidate_keywords_nparray = np.array(bigram_candidate_keywords) 

bigram_candidate_keywords_nparray

keywords_np

tup_nearest_neighbor = []
for index, candidate_keyword in enumerate(keywords_np):
    # Take the current index of the keyword and get the list of 10 nearest index from KNN algorithm
    nearest_neighbors_indeces_of_current_keyword = indeces[index]

    # Filter the keyword list using the list of indeces obtained in previous step
    nearest_keywords = keywords_np[nearest_neighbors_indeces_of_current_keyword]

    # Create tuple with first element as the keyword for current iteration and 2nd element as list of its nearest neighbors
    tup_nearest_neighbor.append((candidate_keyword, nearest_keywords))


tup_nearest_neighbor

[('incorrect', array(['incorrect', 'approaches', 'mappings', 'database',
         'associative databases', 'robot', 'economize', 'juts',
         'organizing associative', 'observes'], dtype='<U24')),
 ('negligible',
  array(['negligible', 'computes', 'handwritten', 'camera', 'parameter',
         'global', 'designing', 'subject', 'assumed', '800'], dtype='<U24')),
 ('free', array(['free', 'recursive', 'hand', 'output', 'candidates',
         'associative databases', 'converges', 'implicitly', 'letter',
         'parameters'], dtype='<U24')),
 ('contrarily', array(['contrarily', 'head letter', 'associative databases',
         'recognizing', 'candidates', 'converged', 'output', 'economize',
         'constructing', 'parameters'], dtype='<U24')),
 ('furthermore',
  array(['furthermore', 'frequency', 'implicitly', 'increases', 'observes',
         'defines', 'global', 'parameters', 'recognizing', 'free'],
        dtype='<U24')),
 ('problem', array(['problem', 'mobile', 'associative datab

# ALL TIME WINDOWS SCI-BERT

In [ ]:
# Convert List of Time Slice DF paper_text content to lists

start = time.time()

# Loop through every DF and convert paper_text to list and concatenate all the papers of one time slice 
## this will be a list like  ["All paper content string of first slice", "all paper content string of 2nd slice", ...] 

papers_contents_list = [" ".join(time_slice_df["paper_text"].tolist()) for time_slice_df in nips_papers_partitions.values()]

#### MEASURE THE EXECUTION TIME FOR RUNNING THE CONCATENATION CODE

end = time.time()
print(end - start)

# papers_contents_list
# len(papers_contents_list[0])

0.06901216506958008


In [ ]:
#  Join Paper titles for bigram and unigram extraction


papers_titles_list = [" ".join(time_slice_df["title"].tolist()) for time_slice_df in nips_papers_partitions.values()]



## Step 1 -  Pre Processing 

# Remove Stopwords 

In [ ]:
import re


# function to rmeove digits and numbers from papers 
def regex_remove_digits(papers_contents_list):      
    # Remove any digits for the corpus
    all_time_window_papers_content_list = [re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", time_slice_paper) 
                                                    for time_slice_paper in papers_contents_list] 
    # Remove words with length less than 3 
    # https://stackoverflow.com/questions/24332025/remove-words-of-length-less-than-4-from-string
    all_time_window_papers_content_list = [re.sub(r'\b\w{1,2}\b', '', time_slice_paper) 
                                          for time_slice_paper in all_time_window_papers_content_list]

    return all_time_window_papers_content_list






In [ ]:

# Custom Stopwords List for Scientific Literature 

from sklearn.feature_extraction import text

path_to_stop_words = '/gdrive/My Drive/Master_dataset/stopwords_10000_most_frequent_filtered.txt'

with open(path_to_stop_words, "r") as file1:
    FileasList = file1.readlines()


stopwords = [s.strip('\n') for s in FileasList]
print(len(stopwords))


scientific_literature_stopwords = text.ENGLISH_STOP_WORDS.union(stopwords)

len(scientific_literature_stopwords)


9954


9958

In [ ]:
# Get all paper content and titles for bigram and unigram generation
all_time_window_papers_content_list = regex_remove_digits(papers_contents_list)
all_time_window_papers_title_list = regex_remove_digits(papers_titles_list)


## Get Bag Of Candidate Keywords For All Time Windows

In [ ]:
# Get Bag of Keywords BiGrams

from sklearn.feature_extraction.text import CountVectorizer
import nltk


def n_gram_generator(all_time_window_papers_content_list, n_gram=1, n_features=200, max_df=0.6, min_df=1):
      
    time_slices_sent_tokens_list_ = [nltk.sent_tokenize(time_window_papers[0:200000]) for time_window_papers in all_time_window_papers_content_list]
    vectorizer = CountVectorizer()

    # time_slices_sent_tokens_list = time_slices_sent_tokens_list_
    
    # time_slice_vectorizer_list = []


    vectorizer_birgram = CountVectorizer(stop_words= scientific_literature_stopwords , ngram_range=(n_gram,n_gram), max_features=n_features, min_df=min_df, max_df=max_df)
    print(vectorizer_birgram)

    x_train_time_slices_list = [vectorizer_birgram.fit_transform(time_slice_sent_tokens) for time_slice_sent_tokens in time_slices_sent_tokens_list_]


    # x_train_time_slices_list = []
    # for time_slice_sent_tokens in time_slices_sent_tokens_list_:
    #         vectorizer_birgram = CountVectorizer(stop_words= scientific_literature_stopwords , ngram_range=(n_gram,n_gram), max_features=n_features) 
    #         x_train_time_slices_list.append(vectorizer_birgram.fit_transform(time_slice_sent_tokens))

    list_of_bow_bi_grams_df_every_time_slice = [pd.DataFrame(x_train.toarray() ,columns=vectorizer_birgram.get_feature_names()) for x_train in x_train_time_slices_list]
    #vect = Dictvectorizer_birgram()


    # x_train = vectorizer_birgram.fit_transform(sent_tokens)

    # df_bow_bi_grams = pd.DataFrame(x_train.toarray(),columns=vectorizer_birgram.get_feature_names())
    # df_bow_bi_grams

    ngram_candidate_keywords_time_slices = [bow_df.columns.values.tolist() for bow_df in list_of_bow_bi_grams_df_every_time_slice]
    # bigram_candidate_keywords = vectorizer_birgram.get_feature_names()
    # len(bigram_candidate_keywords_time_slices)

    feature_frequencies_list_every_time_slice = [x_train.toarray().sum(axis=0) for x_train in x_train_time_slices_list]

    
    # print(feature_frequencies_list_every_time_slice)
    
    ngram_keyword_frequency_list_all_time = []
    for index in range(len(ngram_candidate_keywords_time_slices)):
      feature_frequency = {}
      for j,keyword in enumerate(ngram_candidate_keywords_time_slices[index]):
          feature_frequency[keyword] = feature_frequencies_list_every_time_slice[index][j]
      
      ngram_keyword_frequency_list_all_time.append(feature_frequency)
      


# feature_frequencies = x_train.toarray().sum(axis=0)


# bi_gram_keyword_frequency = {}

# for index, keyword in enumerate(vectorizer.get_feature_names()):
#   bi_gram_keyword_frequency[keyword]=feature_frequencies[index]


    return ngram_keyword_frequency_list_all_time, time_slices_sent_tokens_list_
    # df_bow_bi_grams = pd.DataFrame(x_train.toarray(),columns=vect.get_feature_names())
    # df_bow_bi_grams








In [ ]:
bigram_candidate_keywords_time_slices, time_slices_sent_tokens_list = n_gram_generator(all_time_window_papers_content_list, 2, 200)
unigram_candidate_keywords_time_slices = n_gram_generator(all_time_window_papers_content_list, 1, 200)



all_time_window_papers_titles = " ".join(all_time_window_papers_title_list)

unigram_candidate_keywords_time_slices_title = n_gram_generator([all_time_window_papers_titles], 1, 200,1.0, 0.00)
bigram_candidate_keywords_time_slices_title = n_gram_generator([all_time_window_papers_titles], 2, 200, 0.6, 1)



CountVectorizer(max_df=0.6, max_features=200, ngram_range=(2, 2),
                stop_words=frozenset({'101', '10th', '11th', '121', '12th',
                                      '143', '1b', '1st', '2', '2d', '2nd',
                                      '3d', '3rd', '4', '4th', '5th', '64',
                                      '6th', '7th', '86', '8th', '8vo', '9th',
                                      'FALSE', 'George', 'I', "I'd", 'Michael',
                                      'a', 'a.m.', ...}))


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['adj', 'ain', 'anglo', 'aren', 'cf', 'ch', 'col', 'cong', 'couldn', 'didn', 'doesn', 'false', 'fashioned', 'father', 'ff', 'friend', 'ft', 'gen', 'george', 'girl', 'god', 'hadn', 'hasn', 'haven', 'icel', 'imp', 'isn', 'king', 'lady', 'law', 'majesty', 'michael', 'mother', 'mustn', 'ne', 'needn', 'oe', 'pp', 'prof', 'shan', 'shouldn', 'son', 'sq', 'th', 've', 'viz', 'wasn', 'weren', 'wife', 'woman', 'wouldn'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


CountVectorizer(max_df=0.6, max_features=200,
                stop_words=frozenset({'101', '10th', '11th', '121', '12th',
                                      '143', '1b', '1st', '2', '2d', '2nd',
                                      '3d', '3rd', '4', '4th', '5th', '64',
                                      '6th', '7th', '86', '8th', '8vo', '9th',
                                      'FALSE', 'George', 'I', "I'd", 'Michael',
                                      'a', 'a.m.', ...}))
CountVectorizer(max_features=200, min_df=0.0,
                stop_words=frozenset({'101', '10th', '11th', '121', '12th',
                                      '143', '1b', '1st', '2', '2d', '2nd',
                                      '3d', '3rd', '4', '4th', '5th', '64',
                                      '6th', '7th', '86', '8th', '8vo', '9th',
                                      'FALSE', 'George', 'I', "I'd", 'Michael',
                                      'a', 'a.m.', ...}))
CountVectorizer(ma

In [ ]:
bigram_candidate_keywords_time_slices_title[0][0]


{'adaptive dynamic': 3,
 'adaptive neural': 4,
 'algorithm neural': 5,
 'algorithm observable': 3,
 'algorithm reinforcement': 3,
 'algorithm sequential': 3,
 'algorithms neural': 4,
 'analog neural': 10,
 'analog vlsi': 31,
 'analogue vlsi': 3,
 'anomaly detection': 3,
 'approximate iteration': 3,
 'asymmetric hebbian': 3,
 'asymptotic convergence': 3,
 'attractor neural': 4,
 'audio visual': 5,
 'auditory cortex': 3,
 'backpropagation networks': 3,
 'bayesian averaging': 3,
 'bayesian modeling': 3,
 'bayesian models': 5,
 'bayesian networks': 10,
 'boosting algorithms': 3,
 'cascade correlation': 3,
 'characterization neural': 3,
 'clustering algorithm': 3,
 'clustering graph': 3,
 'competitive neural': 5,
 'computational power': 4,
 'connectionist models': 6,
 'connectionist networks': 3,
 'context free': 3,
 'decoding neuronal': 3,
 'dirichlet allocation': 4,
 'dynamic bayesian': 4,
 'dynamic environments': 3,
 'dynamic models': 4,
 'dynamic programming': 14,
 'dynamics bayesian': 

In [ ]:

# Paper Title Bigrams and Unigrams    
bigram_candidate_keywords_time_slices_title
unigram_candidate_keywords_time_slices_title


# Paper Content Bigrams and Unigrams
unigram_candidate_keywords_time_slices
bigram_candidate_keywords_time_slices


## Create ngrams that include both bigram and unigram for all time windows.
paper_title_ngram_candidate_keywords_time_slice = []

# Loop through the 11 time windows and for each time window create a dictionary of ngrams containing both bi and unigram
paper_title_ngram_candidate_keywords_time_slice.append({**bigram_candidate_keywords_time_slices_title[0][0], **unigram_candidate_keywords_time_slices_title[0][0]})


In [ ]:
# Sort all the keywords for every time slice
title_ngram_candidate_keywords_time_slices_sorted = [sorted(word_frequency.items(),
        key=lambda item: item[1], reverse=True) for word_frequency in paper_title_ngram_candidate_keywords_time_slice]


title_ngram_candidate_keywords_time_slices_sorted[:100]



[[('neural', 425),
  ('networks', 338),
  ('models', 179),
  ('bayesian', 123),
  ('visual', 98),
  ('algorithm', 91),
  ('reinforcement', 86),
  ('gaussian', 85),
  ('clustering', 84),
  ('linear', 84),
  ('algorithms', 83),
  ('kernel', 78),
  ('adaptive', 77),
  ('markov', 72),
  ('neurons', 71),
  ('multi', 69),
  ('vector', 69),
  ('analog', 66),
  ('modeling', 66),
  ('dynamic', 64),
  ('vlsi', 58),
  ('regression', 57),
  ('recurrent', 56),
  ('sparse', 56),
  ('detection', 55),
  ('dynamics', 55),
  ('generalization', 54),
  ('probabilistic', 54),
  ('optimal', 53),
  ('propagation', 51),
  ('connectionist', 47),
  ('hierarchical', 46),
  ('stochastic', 43),
  ('kernels', 42),
  ('supervised', 42),
  ('bounds', 41),
  ('multiple', 41),
  ('optimization', 41),
  ('prediction', 41),
  ('nonlinear', 39),
  ('spike', 39),
  ('statistical', 38),
  ('convergence', 37),
  ('cortex', 37),
  ('spiking', 37),
  ('unsupervised', 37),
  ('gradient', 36),
  ('approximate', 35),
  ('classifi

In [ ]:
# Generate BERT Embeddings for all time windows

from sentence_transformers import SentenceTransformer


model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# List of Candidate Keywords [ [1st time slice candidate keywords], [2nd time slice candidate keywords], .....]
content_time_slices_sent_tokens_list = time_slices_sent_tokens_list


paper_content_bert_embeddings_all_time_slices_list = [ model.encode(time_slice_content[:100000]) for time_slice_content in content_time_slices_sent_tokens_list ]



In [ ]:
len(paper_content_bert_embeddings_all_time_slices_list)

11

In [ ]:
# Get BERT EMbeddings for title ngrams
title_ngrams_bert_embeddings_all_time_slices_list = [ model.encode([tup[0] for tup in time_slice_ngrams]) for time_slice_ngrams in title_ngram_candidate_keywords_time_slices_sorted ]


title_ngrams_bert_embeddings_all_time_slices_list

In [ ]:
title_ngrams_bert_embeddings_all_time_slices_list = title_ngrams_bert_embeddings_all_time_slices_list[0]

title_ngrams_bert_embeddings_all_time_slices_list

In [ ]:
len(title_ngrams_bert_embeddings_all_time_slices_list)

400

In [ ]:
# Save Trained Variables in Google Drive as pickle

saved_map = {
    "paper_content_bert_embeddings_all_time_slices_list" : paper_content_bert_embeddings_all_time_slices_list,
    "title_ngrams_bert_embeddings_all_time_slices_list" : title_ngrams_bert_embeddings_all_time_slices_list
}


import pickle
with open('/gdrive/My Drive/Master_dataset/variables.pickle', 'wb') as f:
     pickle.dump(saved_map, f)




In [ ]:
# Drop the map from globals.
del saved_map

In [ ]:
# Load the pickled variable saved in Drive.
import pickle
with open('/gdrive/My Drive/Master_dataset/variables.pickle', 'rb') as f:
  saved_map = pickle.load(f)

paper_content_bert_embeddings_all_time_slices_list_ = saved_map["paper_content_bert_embeddings_all_time_slices_list"]

In [ ]:
len(paper_content_bert_embeddings_all_time_slices_list_[0])

2298

## Step 3 : Calculate Cosine Similarity of Candidate Keyowrds

In [ ]:
# Create Textual Similarity b/w corpus and candidate keywords
from sentence_transformers import util

cosine_scores = util.cos_sim(title_ngrams_bert_embeddings_all_time_slices_list, title_ngrams_bert_embeddings_all_time_slices_list)

cosine_scores_list = paper_content_bert_embeddings_all_time_slices_list_

top_n = 200
# keywords = [candidate_keywords_bow[index] for index in cosine_scores.argsort()[0][-top_n:]]
keywords


# cosine_scores.argsort()[0][-top_n:]
cosine_scores.argsort()


tensor([[343, 332, 344,  ..., 288, 220,   0],
        [237, 257, 258,  ..., 355, 352,   1],
        [243, 370, 257,  ...,  18, 343,   2],
        ...,
        [170, 344, 300,  ..., 224, 398, 397],
        [243, 344, 188,  ..., 224, 397, 398],
        [258, 386,  23,  ..., 215, 359, 399]])

### TEST 3 - BERT EMBEDDING GENERATE

In [ ]:
title_ngram_candidate_keywords_time_slices_sorted = title_ngram_candidate_keywords_time_slices_sorted[0]

In [ ]:
title_ngram_candidate_keywords_time_slices_sorted

[('neural', 425),
 ('networks', 338),
 ('models', 179),
 ('bayesian', 123),
 ('visual', 98),
 ('algorithm', 91),
 ('reinforcement', 86),
 ('gaussian', 85),
 ('clustering', 84),
 ('linear', 84),
 ('algorithms', 83),
 ('kernel', 78),
 ('adaptive', 77),
 ('markov', 72),
 ('neurons', 71),
 ('multi', 69),
 ('vector', 69),
 ('analog', 66),
 ('modeling', 66),
 ('dynamic', 64),
 ('vlsi', 58),
 ('regression', 57),
 ('recurrent', 56),
 ('sparse', 56),
 ('detection', 55),
 ('dynamics', 55),
 ('generalization', 54),
 ('probabilistic', 54),
 ('optimal', 53),
 ('propagation', 51),
 ('connectionist', 47),
 ('hierarchical', 46),
 ('stochastic', 43),
 ('kernels', 42),
 ('supervised', 42),
 ('bounds', 41),
 ('multiple', 41),
 ('optimization', 41),
 ('prediction', 41),
 ('nonlinear', 39),
 ('spike', 39),
 ('statistical', 38),
 ('convergence', 37),
 ('cortex', 37),
 ('spiking', 37),
 ('unsupervised', 37),
 ('gradient', 36),
 ('approximate', 35),
 ('classifiers', 35),
 ('coding', 33),
 ('mixtures', 33),
 (

In [ ]:
title_ngram_candidate_keywords_time_slices_sorted_ = [ngram[0] for ngram in title_ngram_candidate_keywords_time_slices_sorted[:100]]
t = "\n".join(title_ngram_candidate_keywords_time_slices_sorted_)
t

'neural\nnetworks\nmodels\nbayesian\nvisual\nalgorithm\nreinforcement\ngaussian\nclustering\nlinear\nalgorithms\nkernel\nadaptive\nmarkov\nneurons\nmulti\nvector\nanalog\nmodeling\ndynamic\nvlsi\nregression\nrecurrent\nsparse\ndetection\ndynamics\ngeneralization\nprobabilistic\noptimal\npropagation\nconnectionist\nhierarchical\nstochastic\nkernels\nsupervised\nbounds\nmultiple\noptimization\nprediction\nnonlinear\nspike\nstatistical\nconvergence\ncortex\nspiking\nunsupervised\ngradient\napproximate\nclassifiers\ncoding\nmixtures\nsemi\nprogramming\nanalog vlsi\napproximation\nboosting\ncomputational\nauditory\ndensity\nvariational\nassociative\ncomplexity\nimplementation\nneuron\nchip\nsemi supervised\ncomponent\ndimensional\ncomputation\ncortical\ngraph\ninvariant\nneuronal\nregularization\nspatial\nspiking neurons\nspectral\ndynamical\nextraction\nminimization\nsilicon\nadaptation\ndiscriminative\ndistributions\norganizing\nsampling\nrecurrent neural\ncomputing\nestimating\ngraphical

In [ ]:
!pip install pytorch-pretrained-bert
!pip install Unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 18.2 MB/s 


In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from collections import OrderedDict
import unidecode
import numpy as np
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
import os
import matplotlib.pyplot as plt
# % matplotlib inline

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Found GPU at: /device:GPU:0


'Tesla T4'

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#Use the pre-trained Base BERT model 
model = BertModel.from_pretrained('bert-base-uncased')
model.cuda()
# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

In [ ]:
# s = "This be a semi supervised string"
# if s.find("semi supervised") != -1:
#     print("Found")
# else:
#     print("Not Found")

# targets = ["pca", "semi supervised"]
# corpus = ["This sentence is an example of pca", "pca is good for dimensionality reduction", "this is good example of semi supervised"]


# index, t_ind = datasets._preprocess(targets, corpus)
# t_ind

OrderedDict([('pca', [0, 1]), ('semi supervised', [2])])

In [ ]:
from collections import OrderedDict
class Data():

    def __getitem__(self, content=None):
         if content!=None:
             self.doc = "".join(content)
         return self.doc
     
    def _preprocess(self,targets,corpus):
        self.index=[]
        self.t_index=OrderedDict()
        for target in targets:
            
            for _,item in enumerate(corpus):
                # if target in item:
                  if item.lower().find(target) != -1:
                # if bool(re.search(target, item)):

                      count_target=item.count(target)
                  #   Avoiding the sentences with multiple occurrences of the target term for the time being###
                      if count_target==1:
                        if target not in self.t_index.keys():
                            self.t_index[target]=[_]
                        else:
                            self.t_index[target].append(_)
                        self.index.append(_)
        return self.index,self.t_index

In [ ]:
'''
LOAD & EXTRACT DATA
'''
import os


# OUTPUT_DIR = root_dir+'Colab Notebooks/Challenge_Semeval/CLUSTERING/English_test/' # the path thatcontains the (Corpus1_text, Corpus2_text, Targets)
# p1 = os.path.join(OUTPUT_DIR, 'ccoha1.txt')
# p2 = os.path.join(OUTPUT_DIR, 'ccoha2.txt')
# t = os.path.join(OUTPUT_DIR, 'targets.txt')

# INPUT_DIR = '.\\evaluation\\semeval2020_ulscd_eng\\'
# p1 = os.path.join(INPUT_DIR, 'corpus1\\ccoha1.txt')
# p2 = os.path.join(INPUT_DIR, 'corpus2\\ccoha2.txt')
# #TARGET_DIR = '.\\targets\\'
# t = os.path.join(INPUT_DIR, 'targets.txt')
# p1='ccoha1.txt'
# p2='ccoha2.txt'
# t='targets.txt'

p1 = nips_papers_partitions[0]["paper_text"].tolist()
p2 = nips_papers_partitions[1]["paper_text"].tolist()
t = t
datasets = Data() 

# doc1 =  ["Sentence1", "Sentence2".....]
doc1=datasets.__getitem__(p1).split('\n')   
doc2=datasets.__getitem__(p2).split('\n')
t1=datasets.__getitem__(t).split('\n')
target_act=[x for x in t1 if len(x)>1]
t1=[x.lower() for x in t1 if len(x)>1]
index1=datasets._preprocess(t1,doc1)
index2=datasets._preprocess(t1,doc2)         
index_t1=index1[1]
index_t2=index2[1]
print('The target words are:',t1)
target_words=t1

print('The index_t1 are ', index_t1)
print('The index_t2 are ', index_t2)


#conversions
target_uni=[unidecode.unidecode(m) for m in t1]
target_toks=[]
# print(target_uni)
for k in t1:
  target_toks.append(tokenizer.tokenize(k))
print('converted target toks',target_toks)

In [ ]:
len(t1)
print(len(index_t1))
print(len(index_t2))
target_toks

In [ ]:
def _pre_bert(doc,index,t):
  
  # index =  index_t1 -> { target_w1: index, target_w2: index2, target_w1 : index5 } -  index = Sentence index in which target word appears
  s=["Not Found"]  
  
  if t in index.keys():
      s=[doc[ind] for ind in index[t]]

  print('len of sentences',len(s))
  l=len(s)
  marked_text = ["[CLS] " + text + " [SEP]" for text in s]
  tokenized_text = [tokenizer.tokenize(m) for m in marked_text]
  
  tokenized_text=[x[:512] if len(x)>512 else x for x in tokenized_text]
  indexed_tokens = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]
  segments_ids = [[1] * len(x) for x in tokenized_text]
  return s,marked_text,tokenized_text,indexed_tokens,segments_ids,l


def _bert_features(tokens_tensor, segments_tensors,tokenized_text):
  # print(len(tokens_tensor[0]))
  with torch.no_grad():
      encoded_layers, _ = model(tokens_tensor.to(device), segments_tensors.to(device))
  # print ("Number of layers:", len(encoded_layers))
  layer_i = 0

  # # print ("Number of batches:", len(encoded_layers[layer_i]))
  batch_i = 0

  # print ("Number of tokens:", len(encoded_layers[layer_i][batch_i]))
  token_i = 0

  # print ("Number of hidden units:", len(encoded_layers[layer_i][batch_i][token_i]))
  # Convert the hidden state embeddings into single token vectors

  # Holds the list of 12 layer embeddings for each token
  # Will have the shape: [# tokens, # layers, # features]
  token_embeddings = [] 

  # For each token in the sentence...
  # tokenized_text=[x for x in tokenized_text if x not in ['_', 'n', '##n','v', '##b']]
  for token_i in range(len(tokenized_text)):
    
    # Holds 12 layers of hidden states for each token 
    hidden_layers = [] 
    
    # For each of the 12 layers...
    for layer_i in range(len(encoded_layers)):
      
      # Lookup the vector for `token_i` in `layer_i`
      vec = encoded_layers[layer_i][batch_i][token_i]
      
      hidden_layers.append(vec)
      
    token_embeddings.append(hidden_layers)

  # Sanity check the dimensions:
  # print ("Number of tokens in sequence:", len(token_embeddings))
  # print ("Number of layers per token:", len(token_embeddings[0]))
  return token_embeddings
# s,marked_text,tokenized_text,indexed_tokens,segments_ids
def _get_embeddings(pre,tg):
  m_embed_full=[]
  # print('len(pre[0])',len(pre[0]))
  # print(tg)
  for _,item in enumerate(pre[0]):
    # Convert inputs to PyTorch tensors
    # print(item)
    token_list=pre[2][_]
    
    tokens_tensor = torch.tensor([pre[3][_]])
    segments_tensors = torch.tensor([pre[4][_]])
    # Predict hidden states features for each layer
    token_embeddings=_bert_features(tokens_tensor, segments_tensors,pre[2][_])
    concatenated_last_4_layers = [torch.cat((layer[-1], layer[-2], layer[-3], layer[-4]), 0) for layer in token_embeddings] # [number_of_tokens, 3072]

    summed_last_4_layers = [torch.sum(torch.stack(layer)[-4:], 0) for layer in token_embeddings] # [number_of_tokens, 768]
    
    #consider the tokenized target
  
    indxs=[]
    # print(token_list)
    for tok in tg:
      '''
      remove -1,-2,-3
      '''
      if tok in token_list:
        if tok not in ['_', 'n', '##n','v', '##b']:
          indxs.append(token_list.index(tok))

    # print('indxs',indxs)
    if len(indxs)==1:
      bert_embed=concatenated_last_4_layers [indxs[0]]
      m_embed_full.append(bert_embed)
    elif len(indxs)>1:
      b_emb=[]
      for ind in indxs:
        b_emb.append(concatenated_last_4_layers[ind])
      bert_embed= torch.sum(torch.stack(b_emb), 0)
      m_embed_full.append(bert_embed)
    # indx=token_list.index(tg.lower())
    # indx = [i for (i, elem) in enumerate(pre[2][_]) if t in elem]
    # print('indx',indx)
    # print(pre[1][_],indx)

    # if len(indx)>0:
    # bert_embed=concatenated_last_4_layers[indx[0]]
    
    # cosine_similarity(summed_last_4_layers[10].reshape(1,-1), summed_last_4_layers[19].reshape(1,-1))[0][0]
    
    
  return  m_embed_full
# For a particular target word,do clustering and find if there is a sense change

#[target_words[1]]
def embeddings_extract(target_words,target_toks,doc1,index_t1,doc2,index_t2):
  t=target_words
  X=[]
  X_C1=[]
  X_C2=[]
  sents_all=[]
  lens1=[]
  lens2=[]
  for k,t in enumerate(target_words) :
    berts=[]
    sents=[]
    print('The target word is',t)    
    
    #get the sentences from corpus c1 and c2 for the specific target word 't'
    
    pre1=_pre_bert(doc1,index_t1,t)

    pre2=_pre_bert(doc2,index_t2,t)
    # lens1.append(pre1[-1])
    # lens2.append(pre2[-1])
    # print(pre1)
    
    sents.extend(pre1[0])
    sents.extend(pre2[0])
    #aggregate all the embeddings
    # s,marked_text,tokenized_text,indexed_tokens,segments_ids

    '''
    Get the embeddings of the targets from corpus 1 and 2
    '''
    b1=_get_embeddings(pre1,target_toks[k])
    print('len of t1',len(b1))
    b2=_get_embeddings(pre2,target_toks[k])
    print('len of t2',len(b2))
    '''
    store the lenghts of no. of sentences extracted for each target word for each corpus
    '''
    lens1.append(len(b1))
    lens2.append(len(b2))
    
    berts.extend(b1)
    berts.extend(b2)
    print('len of each target word extractions is',len(berts))
    X.append(berts)
    X_C1.append(b1)# the embeddings for C1
    X_C2.append(b2)#embeddings for C2
    sents_all.append(sents)
  return X,X_C1,X_C2,lens1,lens2,sents_all

In [ ]:
import time
start_time = time.time()

embed_full,embed_C1,embed_C2,len_c1,len_c2,sents=embeddings_extract(target_words,target_toks,doc1,index_t1,doc2,index_t2)

lens=[len_c1]
lens.append(len_c2)
print('saved')
print("--- %s seconds ---" % (time.time() - start_time))

In [135]:
len(embed_C1)

100

In [ ]:
# # target_words.index("semi supervised")
# import re

# print(len(index_t1.keys()))

# print(list(set(t1) - set(index_t1.keys())))
# # if bool(re.search("semi supervised", " ".join(title_ngrams_bert_embeddings_all_time_slices_list[0])) ):
# #   print("as")


# if "semi supervised" in t1:
#   print("found")

97
['pca', 'semi supervised', 'markov models']
found
